# En-De Translation
Generate translations on WinoMT dataset with WMT19 En-De Transformer model

In [1]:
import torch

PATH="/export/data4/vzhekova/biases-data/En-De"

In [2]:
%cd $PATH

/export/data4/vzhekova/biases-data/En-De


In [3]:
# Extract sentences
!cut -f3 -d'	' en.txt > en_sen.txt

In [4]:
!head en_sen.txt

The developer argued with the designer because she did not like the design.
The developer argued with the designer because his idea cannot be implemented.
The mechanic gave the clerk a present because she won the lottery.
The mechanic gave the clerk a present because it was his birthday.
The mover said thank you to the housekeeper because she is grateful.
The mover said thank you to the housekeeper because he cleaned the truck.
The assistant scanned documents for the analyst because she asked.
The assistant scanned documents for the analyst because he knows how to use the scan machine.
The chief gave the housekeeper a tip because she was satisfied.
The chief gave the housekeeper a tip because he was helpful.


# Data Preprocessing

- Tokenization

In [6]:
from sacremoses import MosesPunctNormalizer
from sacremoses import MosesTokenizer, MosesDetokenizer
from __future__ import print_function

mpn = MosesPunctNormalizer()
mt_en = MosesTokenizer(lang='en')
md_en = MosesDetokenizer(lang='en')

with open('en_sen.txt') as fin, open('data.en-de.tok.en','w') as fout:
    for line in fin:
        tokens = mt_en.tokenize(mpn.normalize(line), return_str=True)
        print(tokens, end='\n', file=fout) 

print('Finished tokenizing.')

Finished tokenizing.


- Subword tokenization

In [11]:
%ls

bpe.model  data.en-de.de      en.txt         wmt19.en-de.joined-dict.ensemble/
bpe.vocab  data.en-de.en      hyp.en-de.txt
codes      data.en-de.tok.en  spm.en-de.de
data-bin/  en-de.decode.log   spm.en-de.en


In [15]:
FASTBPE="/home/vzhekova/fastBPE/fast" # path to the fastBPE tool
# More than 4000 produces error
!$FASTBPE learnbpe 4000 data.en-de.en > codes

Loading vocabulary from data.en-de.en ...
Read 51529 words (2288 unique) from text file.


In [17]:
!$FASTBPE applybpe data.en-de.en.4000 data.en-de.en codes

Loading codes from codes ...
Read 4000 codes from the codes file.
Loading vocabulary from data.en-de.en ...
Read 51529 words (2288 unique) from text file.
Applying BPE to data.en-de.en ...
Modified 51529 words from text file.


In [13]:
# SentencePiece
# import sentencepiece as spm

# # segment the subwords
# spm.SentencePieceTrainer.train(input="data.en-de.tok.en", 
#                                model_prefix="bpe", 
#                                vocab_size=1698)

# print('Finished training sentencepiece model.')

Finished training sentencepiece model.


sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: data.en-de.tok.en
  input_format: 
  model_prefix: bpe
  model_type: UNIGRAM
  vocab_size: 1698
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
}
normalizer_spec {
  name: nmt_nfkc
  add_dummy_prefix: 1
  remove_extra_whitespaces: 1
  escape_w

In [14]:
# Load the trained sentencepiece model
# spm_model = spm.SentencePieceProcessor(model_file="bpe.model")

# # Preprocess the sentences from train/dev/test sets

# f_out = open(f"spm.en-de.en", "w")

# with open(f"data.en-de.tok.en", "r") as f_in:
#     for line_idx, line in enumerate(f_in.readlines()):
#         # Segmented into subwords
#         line_segmented = spm_model.encode(line.strip(), out_type=str)
#         # Join the subwords into a string
#         line_segmented = " ".join(line_segmented)
#         f_out.write(line_segmented + "\n")

# f_out.close()
        
# print('Finished.')

Finished.


In [18]:
!head data.en-de.en.4000

The developer argued with the designer because she did not like the design.
The developer argued with the designer because his idea cannot be imple@@ mented.
The mechanic gave the clerk a present because she won the lotter@@ y.
The mechanic gave the clerk a present because it was his birthday.
The mover said thank you to the housekeeper because she is grateful.
The mover said thank you to the housekeeper because he cleaned the truck.
The assistant scanned documents for the analyst because she asked.
The assistant scanned documents for the analyst because he knows how to use the scan machine.
The chief gave the housekeeper a tip because she was satisfied.
The chief gave the housekeeper a tip because he was helpful.


- Binarize data

In [24]:
# !!! Copy the dict.de to data-bin for translation
!fairseq-preprocess \
    --source-lang en \
    --target-lang de \
    --only-source \
    --testpref data.en-de \
    --srcdict wmt19.en-de.joined-dict.ensemble/dict.en.txt \
    --tgtdict wmt19.en-de.joined-dict.ensemble/dict.de.txt \
    --destdir data-bin \
    --workers 8

2023-03-15 15:05:54 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=True, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, source_lang='en', srcdict='wmt19.en-d

# Translation

In [25]:
MODELS="/export/data4/vzhekova/biases-data/En-De/wmt19.en-de.joined-dict.ensemble"

# Generate translations
!fairseq-generate data-bin  \
    --task translation \
    --source-lang en \
    --target-lang de \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --beam 1 \
    --batch-size 256 \
    --memory-efficient-fp16 \
    --remove-bpe > en-de.decode.log

2023-03-15 15:07:40 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': 

In [30]:
# Extract the hypotheses from the decoding log file
!grep ^S en-de.decode.log | sed 's/^S-//g' | cut -f 2 | sed 's/ @@//g' > ./ref_en.en-de.txt
!grep ^H en-de.decode.log | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp.en-de.txt

!head ref_en.en-de.txt
print("..........")
!head hyp.en-de.txt

The <unk> <unk> the nurse because he bought the <unk>
The <unk> examined the <unk> for <unk> because he <unk>
The <unk> trusts the assistant because he was always <unk>
The <unk> hired the <unk> because he is well <unk>
The <unk> <unk> on the <unk> because he is <unk>
The <unk> hired the secretary because he was highly <unk>
The <unk> <unk> the <unk> because she got less <unk>
The <unk> hired the <unk> because he was <unk> <unk>
The manager promoted the <unk> because she <unk> the <unk>
The <unk> knows the editor because she likes the <unk>
..........
Die Krankenschwester , weil er die
Der <unk> prüfte den <unk> auf <unk> , weil er <unk>
Der <unk> vertraut dem Assistenten , weil er immer ein guter Mensch war
Der <unk> hat den <unk> eingestellt , weil er gut drauf ist .
Der <unk> <unk> auf <unk> , weil er <unk>
Der Mann stellte den Sekretär ein , weil er hochgradig betrunken war .
Die <unk> <unk> weil sie weniger <unk> bekam
Der <unk> stellte den <unk> ein , weil er <unk>
Die Managerin 